In [3]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

from keras import backend as K
import keras
import os, re, sys
import pandas as pd
from keras.utils import to_categorical
import ast
from sklearn.utils import class_weight
from keras.preprocessing import sequence
from keras.models import load_model
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
from sklearn.preprocessing import normalize
import time
from keras.models import load_model
from keras import backend as K
import keras
'''
Compatible with tensorflow backend
gamma entre más alto más tolerante (queremos un gamma chico pero no demasiado! 0.25 - 5)
alpha pesos por clase (weights)
'''
def focal_loss(gamma=2., weights=1):   #weights np.asarray()
    weights= K.variable(weights)
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(),1)
        y_pred = K.clip(y_pred,K.epsilon(),1)
        return - K.sum(weights* K.pow(1. - y_pred, gamma)* y_true * K.log(y_pred), axis=-1) 
    return focal_loss_fixed

keras.losses.focal_loss=keras.losses.MSE
keras.losses.focal_loss_fixed=keras.losses.MSE


In [4]:
#'baselines_4/cnn1_ce10_100.h5'
print ("Nh\t\t| IH\t\t| PH\t\t| SH\t\t| F-Macro")
print (0.929514,'\t',0.0000,'\t\t',0.0000,'\t\t',0.84362934,'\t', 0.443285836068) 
#('baselines_4/cnn1_100_focal20_100.h5')
print (0.90495737,'\t',  0.16783217,'\t',  0.11510791 ,'\t', 0.64419476,'\t',0.458023052741)
#'baselines_4/cnn1_100_focal20_100_g2.h5'
print (0.9201278,'\t',   0.0625 ,'\t',     0.05128205 ,'\t', 0.77563452,'\t', 0.452386091144)
#'baselines_4/cnn1_100_focal20_100_g3.h5
print (0.8859146 ,'\t',  0.18723404  ,'\t',0.08510638 ,'\t', 0.6746988,'\t',0.458238453999)
#('baselines_4/rnn1_100_focal15.h5')
print (0.88498746 ,'\t', 0.09722222,'\t',  0.13903743 ,'\t', 0.79787234,'\t',  0.479779863874)
#('baselines_4/rnn2_100_focal15.h5')
print ( 0.8779625 ,'\t',  0.14334471,'\t',  0.08130081  ,'\t',0.80635551,'\t',0.477240884687)

Nh		| IH		| PH		| SH		| F-Macro
0.929514 	 0.0 		 0.0 		 0.84362934 	 0.443285836068
0.90495737 	 0.16783217 	 0.11510791 	 0.64419476 	 0.458023052741
0.9201278 	 0.0625 	 0.05128205 	 0.77563452 	 0.452386091144
0.8859146 	 0.18723404 	 0.08510638 	 0.6746988 	 0.458238453999
0.88498746 	 0.09722222 	 0.13903743 	 0.79787234 	 0.479779863874
0.8779625 	 0.14334471 	 0.08130081 	 0.80635551 	 0.477240884687


In [5]:
cnn1_64 = load_model('baselines_4/cnn1_ce10_100.h5')
cnn1_100 = load_model('baselines_4/cnn1_100_focal20_100.h5')
cnn1_f2 = load_model('baselines_4/cnn1_100_focal20_100_g2.h5')
cnn1_f3 = load_model('baselines_4/cnn1_100_focal20_100_g3.h5')
gru1 = load_model('baselines_4/rnn1_100_focal15.h5')
gru2 = load_model('baselines_4/rnn2_100_focal15.h5')


list_models=['cnn1_64', 'cnn1_100', 'cnn1_f2', 'cnn1_f3', 'gru1', 'gru2']
index_models=np.arange(6)
dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
modelos=[cnn1_64, cnn1_100, cnn1_f2, cnn1_f3, gru1, gru2]
ind=np.arange(6)
dict_trainingModel=dict((key, value) for (key, value) in zip(ind,modelos))

In [7]:
predicciones_all=[]
X_train=np.load("matrices/X_train_4.npy")
X_train_rnn=np.load("matrices/X_train_rnn_4.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='gru':
        predicciones_all.append(dict_trainingModel[i].predict(X_train, batch_size=bs))
    else:
        predicciones_all.append(dict_trainingModel[i].predict(X_train_rnn, batch_size=bs))
        
predicciones_all_val=[]
X_val=np.load("matrices/X_val_ig_4.npy")
X_val_rnn=np.load("matrices/X_val_rnn_ig_4.npy")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones val del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='gru':
        predicciones_all_val.append(dict_trainingModel[i].predict(X_val, batch_size=bs))
    else:
        predicciones_all_val.append(dict_trainingModel[i].predict(X_val_rnn, batch_size=bs))
        

Agregando predicciones del modelo cnn1_64
Agregando predicciones del modelo cnn1_100
Agregando predicciones del modelo cnn1_f2
Agregando predicciones del modelo cnn1_f3
Agregando predicciones del modelo gru1
Agregando predicciones del modelo gru2
Agregando predicciones val del modelo cnn1_64
Agregando predicciones val del modelo cnn1_100
Agregando predicciones val del modelo cnn1_f2
Agregando predicciones val del modelo cnn1_f3
Agregando predicciones val del modelo gru1
Agregando predicciones val del modelo gru2


In [ ]:
predicciones_all_test=[]
X_test=np.load("")
X_test_rnn=np.load("")

bs=32
for i in dict_trainingModel.keys(): 
    print ("Agregando predicciones test del modelo", dict_models[i])
    a_evaluar=dict_models[i]
    if a_evaluar[:3]!='gru':
        predicciones_all_test.append(dict_trainingModel[i].predict(X_test, batch_size=bs))
    else:
        predicciones_all_test.append(dict_trainingModel[i].predict(X_test_rnn, batch_size=bs))

In [8]:
matrix=np.asarray(predicciones_all)
print (matrix.shape)
matrix_val=np.asarray(predicciones_all_val)
print (matrix_val.shape)
#matrix_test=np.asarray(predicciones_all_test)
#print (matrix_test.shape)

(6, 6374, 4)
(6, 2125, 4)


In [9]:
new_matrix=np.concatenate([matrix,matrix_val],axis=1)#,matrix_test], axis=1)
new_matrix.shape

(6, 8499, 4)

In [10]:
Ks=np.load("../Orthogonals_768.npy")
print (Ks.shape)
random=np.random.randint(Ks.shape[0], size=4)
K_list=[Ks[k] for k in random]
print (np.asarray(K_list).shape)

(768, 768)
(4, 768)


In [12]:
class Dummy_Embeddings(nn.Module):
    def __init__(self, d_model, dummy_vectors, dict_ind_emb): 
        super(Dummy_Embeddings, self).__init__()
        aux = torch.from_numpy(np.asarray(dummy_vectors))
        self.index2dummy = nn.Embedding(aux.size()[0], d_model)
        self.index2dummy.weigth=nn.Parameter(aux)
        self.index2dummy.weigth.requires_grad=False
        self.d_model = d_model
        
    def forward(self, x):
        aux=x.data
        max_a=aux.size()[0]
        max_b=aux.size()[1]
        new_x= np.zeros((max_a,max_b),dtype='int32')
        for obj in range(max_a):
            for elem in range(max_b):
                #print ("en dummy embedding x.data[obj][elem]",x.data[obj][elem].item())
                new_x[obj,elem] = dict_ind_emb[x.data[obj][elem].item()] - 1
        return self.index2dummy(torch.from_numpy(new_x).long()) * math.sqrt(self.d_model)
    
def match(objetos,ejemplo):
    i=0
    for obj in objetos:
        if obj==ejemplo:
            return i
        i+=1
        
class EncoderDecoderLogSoft(nn.Module):
    def __init__(self, encoder, src_embed, sequential):
        super(EncoderDecoderLogSoft, self).__init__()
        self.encoder = encoder
        self.linearSoft = sequential
        self.src_embed = src_embed
        
    def forward(self, src, src_mask):
        out=self.encode(src, src_mask)
        retorno=self.toSoftmax(out)
        return retorno
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def toSoftmax(self, tensor):
        a=tensor.size()[0]
        b=tensor.size()[-1]
        c=tensor.size()[-2]
        new_tensor= np.zeros((a, b))
        ini=True
        for ai in range(a):
            for ci in range(c):
                if ini:
                    new_tensor[ai]=tensor.data[ai][ci].numpy()
                    ini=False
                else: 
                    new_tensor[ai]*=tensor.data[ai][ci].numpy()
            ini=True
        new_tensor=torch.from_numpy(new_tensor)
        lineal=self.linearSoft(new_tensor.float())
        return F.log_softmax(lineal, dim = -1)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class EncoderDecoderSoft(nn.Module):
    def __init__(self, encoder, src_embed, sequential):
        super(EncoderDecoderSoft, self).__init__()
        self.encoder = encoder
        self.linearSoft = sequential
        self.src_embed = src_embed
        
    def forward(self, src, src_mask):
        out=self.encode(src, src_mask)
        retorno=self.toSoftmax(out)
        return retorno
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def toSoftmax(self, tensor):
        a=tensor.size()[0]
        b=tensor.size()[-1]
        c=tensor.size()[-2]
        new_tensor= np.zeros((a, b))
        ini=True
        for ai in range(a):
            for ci in range(c):
                if ini:
                    new_tensor[ai]=tensor.data[ai][ci].numpy()
                    ini=False
                else: 
                    new_tensor[ai]*=tensor.data[ai][ci].numpy()
            ini=True
            
        new_tensor=torch.from_numpy(new_tensor)
        lineal=self.linearSoft(new_tensor.float())
        return F.softmax(lineal, dim = -1)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, src_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.generator = generator
        
    def forward(self, src, src_mask):
        return  self.encode(src, src_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)
    
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))    
    
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)
    
def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
         
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
    
class PositionalText(nn.Module):
    def __init__(self, d_model, dropout, max_len=5):
        super(PositionalText, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x):
        x_text=x.data[:,0][:,None,:]
        x_machines=x.data[:,1:]
        
        x_return = x_machines* x_text
        return self.dropout(x_return)
    
class PositionalText_concat(nn.Module):
    def __init__(self, d_model, dropout, max_len=5):
        super(PositionalText_concat, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x):
        return self.dropout(x)

In [13]:
def data_gen(batch_size,mode,n_machines,new_matrix):    
    x_data=[] 
    y_data=[]
    if mode=='Train':
        max_objs=matrix.shape[1]
        x_list=np.arange(max_objs)
        np.random.shuffle(x_list)
        objs=np.load("obj_ID_train.npy")
        labels=np.load("../Numpy_matrix/y_train_models.npy")        
    if mode=='Val':
        max_objs=matrix_val.shape[1] 
        x_list=np.arange(max_objs)
        x_list=[x+matrix.shape[1] for x in x_list]
        np.random.shuffle(x_list)
        objs=np.load("obj_ID_val.npy")
        labels=np.load("../Numpy_matrix/y_val_models.npy")
    if mode=='Test':
        max_objs=matrix_test.shape[1] 
        x_list=np.arange(max_objs)
        x_list=[x+matrix.shape[1]+matrix_val.shape[1] for x in x_list]
        np.random.shuffle(x_list)
        objs=np.load("obj_ID_test.npy")#np.array(np.load("obj_ID_test.npy")[0])
        labels=np.load("y_test_models.npy")#np.array(np.load("y_test_models.npy")[0])
        
    if len(x_list)%batch_size==0:
        n_batches=len(x_list)/batch_size
        for nb in range(n_batches): 
            x_data.append(x_list[nb*batch_size:(nb+1)*batch_size])
    else:
        n_batches=int(len(x_list)/batch_size)
        for nb in range(n_batches): 
            x_data.append(x_list[nb*batch_size:(nb+1)*batch_size])
    
    new_x_data=[] 
    if mode=='Train':
        for batch in x_data:
            temp=[]
            for exam in batch:
                temp2=[objs[exam]] ### obj evaluado en maquinas
                for m in range(n_machines): 
                    temp2= temp2 + [np.argmax(new_matrix[m][exam])+1]   ###temp2 = [tweetID, index_class]
                temp.append(temp2)
            new_x_data.append(temp) 
            
    if mode=='Val':
        for batch in x_data:
            temp=[]
            for exam in batch:
                temp2=[objs[exam-matrix.shape[1]]]                      ### obj evaluado en maquinas
                for m in range(n_machines): 
                    temp2= temp2 + [np.argmax(new_matrix[m][exam])+1]   ###temp2 = [tweetID, index_class]
                temp.append(temp2)
            new_x_data.append(temp)
            
    if mode=='Test':
        for batch in x_data:
            temp=[]
            for exam in batch:
                temp2=[objs[exam-matrix.shape[1]-matrix_val.shape[1]]]  ### obj evaluado en maquinas
                for m in range(n_machines): 
                    temp2= temp2 + [np.argmax(new_matrix[m][exam])+1]   ###temp2 = [tweetID, index_class]
                temp.append(temp2)
            new_x_data.append(temp)

    y_data=[]
    for batch in new_x_data:
        temp_y=[]
        for exam in batch:
            q=exam[0]
            indice=match(objs, q)    #retorna indice de match tweetID==objs_i
            temp_y.append([labels[indice]+1])       #appendear grounnf truth de tweetID
        y_data.append(temp_y)
        

    new_x_data=np.asarray(new_x_data)
    y_data=np.asarray(y_data)
    
    for x,y in zip(new_x_data, y_data):
        src = Variable(torch.from_numpy(np.asarray(x)), requires_grad=False)
        stances = Variable(torch.from_numpy(np.asarray(y)), requires_grad=False)
        yield Batch(src, stances, 0)

In [ ]:
K_list_ext=K_list
objs_train=np.load("obj_ID_train.npy")
objs_val=np.load("obj_ID_val.npy")
objs_test=np.load("obj_ID_test.npy")
print (objs_train.shape)
print (objs_val.shape)
print (objs_test.shape)
llaves=np.concatenate([objs_train,objs_val,objs_test])
valores=np.arange(5, objs_train.shape[0]+objs_val.shape[0]+objs_test.shape[0]+5)
len(valores)

In [ ]:
dict_IDObj=dict((key, value) for (key, value) in zip(llaves,valores))
dict_ObjID=dict((key, value) for (key, value) in zip(valores,llaves))

In [ ]:
len(dict_ObjID)

In [ ]:
print (len(K_list_ext))
for indice in dict_ObjID.keys():
    try:
        vector=np.load("NumpyVectors/"+str(dict_ObjID[indice])+".npy")
        K_list_ext.append(vector)
        print ("Cargando desde Train-Val")
    except:
        vector=np.load("../NumpyTest/"+str(dict_ObjID[indice])+".npy")
        K_list_ext.append(vector)
        print ("Cargando desde Test")

In [ ]:
len(K_list_ext)

In [ ]:
dict_stn=dict()
dict_stn[1]=1
dict_stn[2]=2
dict_stn[3]=3
dict_stn[4]=4
dict_stn

In [ ]:
#from itertools import chain

#dict_ind_emb=dict(chain.from_iterable(d.iteritems() for d in (dict_stn, dict_IDObj)))
dict_ind_emb = {**dict_stn, **dict_IDObj}
dict_ind_emb

In [ ]:
def make_model(target, N=2, d_model=768, d_ff=1024, h=4, dropout=0.3):  #recibir src_vocab si lo utiliza dummy_Emb
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalText(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        nn.Sequential(Dummy_Embeddings(768, K_list_ext, dict_ind_emb), c(position)),
        Generator(d_model, target))
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

def make_model_concat(target, N=2, d_model=768, d_ff=1024, h=4, dropout=0.3, tipo='Soft'):  #recibir src_vocab si lo utiliza dummy_Emb
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalText_concat(d_model, dropout)
    if tipo=='log':
        print ("")
        print ("Creando modelo con salida Log_softmax")
        model = EncoderDecoderLogSoft(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        nn.Sequential(Dummy_Embeddings(768, K_list_ext, dict_ind_emb), c(position)),
        nn.Linear(d_model, 4))
    else:
        print ("Creando modelo con salida Softmax")
        model = EncoderDecoderSoft(
            Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
            nn.Sequential(Dummy_Embeddings(768, K_list_ext, dict_ind_emb), c(position)),
            nn.Linear(d_model, 4))
    
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

class Batch:
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :]
            self.trg_y = trg[:, 0:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
            
    @staticmethod
    def make_std_mask(tgt, pad):
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask
    
def run_epoch(data_iter, model, loss_compute, mode):
    #print ("en data iter de run epoch")
    if mode=='Test':
        with torch.no_grad():
            start = time.time()
            total_datos = 0
            total_loss = 0
            total_acc=0
            total_fm1=0
            total_fs=0
            n_batches = 0
            cm_last=np.zeros((4,4))
            for i, batch in enumerate(data_iter):
                out = model.forward(batch.src, batch.src_mask)
                loss,acc,f1,cm = loss_compute(out, batch.trg_y, batch.ntokens, mode)
                #la loss del batch es sobre todos los ejemplos o es promedio en batch?
                
                cm_last+=cm
                f_ma=np.mean(f1)
                total_loss += loss
                """print ("total_loss", total_loss)
                print ("loss del batch", loss)
                print ("Acc de batch",acc)
                print ("F1 de batch",f1)"""
                total_acc += np.array(acc)
                total_fm1 += np.sum(f1)/4.0
                total_fs += f1
                total_datos += batch.ntokens
                n_batches += 1
                #if i % 50 == 0:    
                #    print("Step: %d Batch Loss: %f Acc: %f F1: %s F1_ma: %f" % (i,loss/n_batches, acc, str(f1), f_ma))
            return total_loss/n_batches, torch.from_numpy(np.array(total_acc))/n_batches, torch.from_numpy(np.array(total_fm1))/n_batches, torch.from_numpy(np.array(total_fs))/n_batches,cm_last
    else:
        start = time.time()
        total_learners = 0
        total_datos = 0
        total_loss = 0
        total_acc=0
        total_fm1=0
        total_fs=0
        n_batches = 0
        cm_last=np.zeros((4,4))
        #print ("training")
        for i, batch in enumerate(data_iter):
            out = model.forward(batch.src, batch.src_mask)
            loss,acc,f1,cm = loss_compute(out, batch.trg_y, batch.ntokens, mode)
            cm_last+=cm
            f_ma=np.mean(f1)
            total_loss += loss
            total_acc += np.array(acc)
            total_fm1 += np.sum(f1)/4.0
            total_fs += f1  ###para train
            total_datos += batch.ntokens
            n_batches += 1
            #if i % 1 == 0:    ##### cambiar a 50 de acuerdo a mis datos. 
            #    print("Epoch Step: %d Loss: %f Acc: %f F1: %s F1_ma: %f" % (i,loss/n_batches, acc, str(f1), f_ma))
        return total_loss/n_batches, torch.from_numpy(np.array(total_acc))/n_batches, torch.from_numpy(np.array(total_fm1))/n_batches, torch.from_numpy(np.array(total_fs))/n_batches, cm_last

In [ ]:
global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of learners + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.src))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.trg) + 2)
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    maximo=max(src_elements, tgt_elements)
    return maximo

class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        #print ("Rate de Noam_opt", self._rate, self._step)
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        mask = torch.nonzero(target.data == self.padding_idx)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))
    
class LabelCCE(nn.Module):
    def __init__(self, class_weights):
        super(LabelCCE, self).__init__()
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)
                       
    def forward(self, x, target):
        return self.criterion(x, target)
    
class FocalLoss(nn.Module):
    def __init__(self, weights, gamma=2.0, reduce=True):#, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.weights = weights
        self.gamma = gamma
        #self.logits = logits
        self.reduce = reduce

    def forward(self, x, y):
        tensors = []
        for l in y:
            tensors.append(torch.zeros(4).scatter_(0, torch.tensor(l), 1))
        result = torch.stack(tensors, 0)
        
        CCE_loss = F.cross_entropy(x, result.long().argmax(dim=-1), reduction='none', weight=self.weights.float())
        pt = torch.exp(-CCE_loss)
        factor= (1-pt)**self.gamma * CCE_loss
        
        F_loss = factor  #*self.weights.float()-- peso esta dentro de la funcion

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
class SimpleLossComputeFL:
    "A simple loss compute and train function."
    def __init__(self, criterion, opt=None):
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm, mode):        
        y_new=(y.float()-torch.ones((y.shape[0],y.shape[1]))).int()
        ac=accuracy_scorer(x, y_new)
        f1=f_scorer(x, y_new)
        cm=compute_confusion_matrix(x, y_new)
        tempa= x.contiguous().view(-1, x.size(-1))
        tempb= y_new.long().contiguous().view(-1)
        loss = self.criterion(tempa, tempb) #/ norm
        if mode!='Test':
            loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return [loss.data, ac ,f1, cm] #*norm #[0] * norm
    
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    

In [ ]:
def accuracy_scorer(predicho, target):
    predicho= torch.argmax(predicho, dim=1)
    return accuracy_score(target.numpy(),predicho.numpy())

def f_scorer(predicho, target):
    predicho= torch.argmax(predicho, dim=1)
    return f1_score(target.numpy(),predicho.numpy(), average=None, labels=[0,1,2,3])

def compute_confusion_matrix(predicho, target):
    predicho= torch.argmax(predicho, dim=1)
    return confusion_matrix(target.numpy(), predicho.numpy(), labels=[0,1,2,3])

y_train_stance=np.load("../Numpy_matrix/y_train_models.npy")
print (y_train_stance[:10])
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train_stance), y_train_stance)#{0: 3.,   1: 6.,   2: 5.,  3: 3.}
print (class_weights)

if not sys.warnoptions:
    warnings.simplefilter("ignore")

## Modelos